In [1]:
import torch
print(torch.version.cuda)

12.4


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.cuda.amp import GradScaler, autocast  # For mixed precision training
import pandas as pd
from sklearn.model_selection import ParameterGrid

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data = pd.read_csv("filtered_specialists.csv")

specialist_mapping = {spec: idx for idx, spec in enumerate(data["Specialist"].unique())}
data["Specialist_id"] = data["Specialist"].map(specialist_mapping)

train_texts, test_texts, train_labels, test_labels = train_test_split(
    data["Patient"].astype(str)+"\n"+data["Description"].astype(str), 
    data["Specialist_id"],
    test_size=0.2,
    random_state=42,
)

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

class SpecialistDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])  
        label = self.labels[idx]
        encoded = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
        )
        return {
            "input_ids": encoded["input_ids"].squeeze(0),
            "attention_mask": encoded["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long),
        }

train_dataset = SpecialistDataset(train_texts.tolist(), train_labels.tolist(), tokenizer)
test_dataset = SpecialistDataset(test_texts.tolist(), test_labels.tolist(), tokenizer)

def collate_fn(batch):
    input_ids = torch.stack([item["input_ids"] for item in batch])
    attention_mask = torch.stack([item["attention_mask"] for item in batch])
    labels = torch.tensor([item["label"] for item in batch])
    return {"input_ids": input_ids, "attention_mask": attention_mask, "label": labels}

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=collate_fn)

class CNNForSpecialistClassification(nn.Module):
    def __init__(self, pretrained_model_name, num_labels, embedding_dim=768, kernel_size=3, num_filters=128):
        super(CNNForSpecialistClassification, self).__init__()
        self.bert = AutoModel.from_pretrained(pretrained_model_name)
        self.bert.requires_grad_(False) 

        self.conv1 = nn.Conv1d(in_channels=embedding_dim, out_channels=num_filters, kernel_size=kernel_size)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.fc = nn.Linear(num_filters, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        embeddings = outputs.last_hidden_state  # Shape: [batch_size, seq_len, embedding_dim]

        # Apply CNN layers
        embeddings = embeddings.transpose(1, 2)  # Change to [batch_size, embedding_dim, seq_len]
        x = self.conv1(embeddings)  # Apply convolutional layer
        x = torch.relu(x)
        x = self.pool(x)  # Apply pooling

        # Global Average Pooling
        x = x.mean(dim=2)  # Average over the sequence length dimension

        # Final classification layer
        x = self.fc(x)
        return x

# Optimizer Setup
def get_optimizer(model, lr=1e-5):
    return optim.Adam(model.parameters(), lr=lr)

# Training Function
def train_model(model, train_loader, optimizer, criterion, device, scaler):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        with autocast():  
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = criterion(outputs, labels)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        total_loss += loss.item()
    return total_loss / len(train_loader)

# Evaluation Function
def evaluate_model(model, test_loader, device):
    model.eval()
    preds = []
    labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels.extend(batch["label"].tolist())

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds.extend(torch.argmax(outputs, dim=1).tolist())
    accuracy = accuracy_score(labels, preds)
    return accuracy

# Hyperparameter tuning using grid search
param_grid = {
    'learning_rate': [1e-5, 5e-5, 1e-4],
    'batch_size': [16, 32, 64],
    'num_filters': [32, 64, 128],
    'kernel_size': [3, 4, 5]
}

# Hyperparameter Grid Search
def hyperparameter_tuning(train_loader, test_loader, param_grid):
    best_accuracy = 0
    best_params = None
    for params in ParameterGrid(param_grid):
        print(f"Training with params: {params}")
        
        model = CNNForSpecialistClassification(
            pretrained_model_name=model_name, 
            num_labels=len(specialist_mapping),
            num_filters=params['num_filters'],
            kernel_size=params['kernel_size']
        ).to(device)
        
        optimizer = get_optimizer(model, lr=params['learning_rate'])
        criterion = nn.CrossEntropyLoss()
        scaler = GradScaler()  
        
        train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True, collate_fn=collate_fn)
        test_loader = DataLoader(test_dataset, batch_size=params['batch_size'], collate_fn=collate_fn)
      
        for epoch in range(3):  # Limiting to 3 epochs for quicker tuning
            train_loss = train_model(model, train_loader, optimizer, criterion, device, scaler)
            accuracy = evaluate_model(model, test_loader, device)
            print(f"Epoch {epoch+1}/3 | Train Loss: {train_loss:.4f} | Test Accuracy: {accuracy:.4f}")

            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = params

    print(f"Best Params: {best_params} with Accuracy: {best_accuracy:.4f}")
    return best_params

best_params = hyperparameter_tuning(train_loader, test_loader, param_grid)

final_model = CNNForSpecialistClassification(
    pretrained_model_name=model_name, 
    num_labels=len(specialist_mapping),
    num_filters=best_params['num_filters'],
    kernel_size=best_params['kernel_size']
).to(device)

final_optimizer = get_optimizer(final_model, lr=best_params['learning_rate'])
final_criterion = nn.CrossEntropyLoss()
final_scaler = GradScaler()

for epoch in range(5):  
    train_loss = train_model(final_model, train_loader, final_optimizer, final_criterion, device, final_scaler)
    accuracy = evaluate_model(final_model, test_loader, device)
    print(f"Epoch {epoch+1}/5 | Train Loss: {train_loss:.4f} | Test Accuracy: {accuracy:.4f}")

# Save the Fine-Tuned Model
torch.save(final_model.state_dict(), "./cnn_fine_tuned_model.pth")
tokenizer.save_pretrained("./cnn_fine_tuned_model_tokenizer")

/home/keshav/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Training with params: {'batch_size': 16, 'kernel_size': 3, 'learning_rate': 1e-05, 'num_filters': 32}


/tmp/ipykernel_136960/1040111746.py:175: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()  # For mixed precision
/tmp/ipykernel_136960/1040111746.py:122: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Automatically cast operations to half precision


Epoch 1/3 | Train Loss: 2.9564 | Test Accuracy: 0.3325


/tmp/ipykernel_136960/1040111746.py:122: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Automatically cast operations to half precision


...

Epoch 3/3 | Train Loss: 1.6151 | Test Accuracy: 0.5389
Training with params: {'batch_size': 16, 'kernel_size': 4, 'learning_rate': 0.0001, 'num_filters': 32}


/tmp/ipykernel_136960/1040111746.py:175: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()  # For mixed precision
/tmp/ipykernel_136960/1040111746.py:122: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Automatically cast operations to half precision


Epoch 1/3 | Train Loss: 2.1280 | Test Accuracy: 0.4959


/tmp/ipykernel_136960/1040111746.py:122: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Automatically cast operations to half precision


KeyboardInterrupt: 

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.cuda.amp import GradScaler, autocast  # For mixed precision training
import pandas as pd

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load CSV
data = pd.read_csv("filtered_specialists.csv")

# Map specialists to numeric labels
specialist_mapping = {spec: idx for idx, spec in enumerate(data["Specialist"].unique())}
data["Specialist_id"] = data["Specialist"].map(specialist_mapping)

# Train-test split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data["Patient"].astype(str) + "\n" + data["Description"].astype(str),  # Convert to string explicitly
    data["Specialist_id"],
    test_size=0.2,
    random_state=42,
)

# Load BERT tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Custom Dataset Class
class SpecialistDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])  # Ensure each text is a string
        label = self.labels[idx]
        encoded = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
        )
        return {
            "input_ids": encoded["input_ids"].squeeze(0),
            "attention_mask": encoded["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long),
        }

# Create Datasets
train_dataset = SpecialistDataset(train_texts.tolist(), train_labels.tolist(), tokenizer)
test_dataset = SpecialistDataset(test_texts.tolist(), test_labels.tolist(), tokenizer)

# Data Collation Function
def collate_fn(batch):
    input_ids = torch.stack([item["input_ids"] for item in batch])
    attention_mask = torch.stack([item["attention_mask"] for item in batch])
    labels = torch.tensor([item["label"] for item in batch])
    return {"input_ids": input_ids, "attention_mask": attention_mask, "label": labels}

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, collate_fn=collate_fn)

# Improved CNN Model
class ImprovedCNNForSpecialistClassification(nn.Module):
    def __init__(self, pretrained_model_name, num_labels, embedding_dim=768, kernel_sizes=[3, 4, 5], num_filters=128, dropout_rate=0.3):
        super(ImprovedCNNForSpecialistClassification, self).__init__()
        self.bert = AutoModel.from_pretrained(pretrained_model_name)
        self.bert.requires_grad_(False) 
    
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=embedding_dim, out_channels=num_filters, kernel_size=k)
            for k in kernel_sizes
        ])
  
        self.batch_norm = nn.BatchNorm1d(num_filters * len(kernel_sizes))

        self.dropout = nn.Dropout(dropout_rate)

        self.fc = nn.Linear(num_filters * len(kernel_sizes), num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        embeddings = outputs.last_hidden_state  # Shape: [batch_size, seq_len, embedding_dim]

        embeddings = embeddings.transpose(1, 2)  # Change to [batch_size, embedding_dim, seq_len]
        conv_outputs = []
        for conv in self.convs:
            conv_output = torch.relu(conv(embeddings))  
            conv_output = torch.max(conv_output, dim=2)[0]  
            conv_outputs.append(conv_output)

        x = torch.cat(conv_outputs, dim=1)  # Shape: [batch_size, num_filters * len(kernel_sizes)]
        x = self.batch_norm(x)
        x = self.dropout(x)
        x = self.fc(x)
        return x

# Initialize the model with the best hyperparameters
model = ImprovedCNNForSpecialistClassification(
    pretrained_model_name=model_name,
    num_labels=len(specialist_mapping),
    num_filters=128,
    kernel_sizes=[3, 4, 5],
    dropout_rate=0.3
).to(device)

# Optimizer Setup
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Loss Function
criterion = nn.CrossEntropyLoss()

# Mixed Precision Training
scaler = GradScaler()

# Training Function
def train_model(model, train_loader, optimizer, criterion, device, scaler):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        # Mixed precision training
        with autocast():  # Automatically cast operations to half precision
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = criterion(outputs, labels)

        # Scaler for mixed precision
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        total_loss += loss.item()
    return total_loss / len(train_loader)

# Evaluation Function
def evaluate_model(model, test_loader, device):
    model.eval()
    preds = []
    labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels.extend(batch["label"].tolist())

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds.extend(torch.argmax(outputs, dim=1).tolist())
    accuracy = accuracy_score(labels, preds)
    return accuracy

# Training Loop
for epoch in range(5):  # Train for 5 epochs
    train_loss = train_model(model, train_loader, optimizer, criterion, device, scaler)
    accuracy = evaluate_model(model, test_loader, device)
    print(f"Epoch {epoch + 1}/5 | Train Loss: {train_loss:.4f} | Test Accuracy: {accuracy:.4f}")

# Save the Fine-Tuned Model
torch.save(model.state_dict(), "./improved_cnn_fine_tuned_model.pth")
tokenizer.save_pretrained("./improved_cnn_fine_tuned_model_tokenizer")

/home/keshav/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_171792/514804329.py:126: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/tmp/ipykernel_171792/514804329.py:139: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Automatically cast operations to half precision


Epoch 1/5 | Train Loss: 1.6830 | Test Accuracy: 0.6593


/tmp/ipykernel_171792/514804329.py:139: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Automatically cast operations to half precision


Epoch 2/5 | Train Loss: 1.2565 | Test Accuracy: 0.6820


/tmp/ipykernel_171792/514804329.py:139: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Automatically cast operations to half precision


...

/tmp/ipykernel_171792/514804329.py:139: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Automatically cast operations to half precision


Epoch 4/5 | Train Loss: 1.0433 | Test Accuracy: 0.6994


/tmp/ipykernel_171792/514804329.py:139: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Automatically cast operations to half precision


Epoch 5/5 | Train Loss: 0.9701 | Test Accuracy: 0.7013


('./improved_cnn_fine_tuned_model_tokenizer/tokenizer_config.json',
 './improved_cnn_fine_tuned_model_tokenizer/special_tokens_map.json',
 './improved_cnn_fine_tuned_model_tokenizer/vocab.txt',
 './improved_cnn_fine_tuned_model_tokenizer/added_tokens.json',
 './improved_cnn_fine_tuned_model_tokenizer/tokenizer.json')

## Bio-Bert Code Final

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.cuda.amp import GradScaler, autocast  # For mixed precision training
import pandas as pd

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load CSV
data = pd.read_csv("filtered_specialists.csv")

speciality_counts = data['Specialist'].value_counts()

# Get specialities that occur more than 1000 times
specialities_to_keep = speciality_counts[speciality_counts > 500].index

# Filter the data to keep only rows with specialities that occur more than 1000 times
data = data[data['Specialist'].isin(specialities_to_keep)]

# Map the filtered specialists to numeric labels
specialist_mapping = {spec: idx for idx, spec in enumerate(data["Specialist"].unique())}
data["Specialist_id"] = data["Specialist"].map(specialist_mapping)

# Train-test split
train_texts, test_texts, train_labels, test_labels = train_test_split(
    data["Patient"].astype(str) + "\n" + data["Description"].astype(str),  # Convert to string explicitly
    data["Specialist_id"],
    test_size=0.2,
    random_state=42,
)

# Load BioBERT tokenizer
model_name = "monologg/biobert_v1.1_pubmed"  # BioBERT model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Custom Dataset Class
class SpecialistDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])  # Ensure each text is a string
        label = self.labels[idx]
        encoded = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
        )
        return {
            "input_ids": encoded["input_ids"].squeeze(0),
            "attention_mask": encoded["attention_mask"].squeeze(0),
            "label": torch.tensor(label, dtype=torch.long),
        }

# Create Datasets
train_dataset = SpecialistDataset(train_texts.tolist(), train_labels.tolist(), tokenizer)
test_dataset = SpecialistDataset(test_texts.tolist(), test_labels.tolist(), tokenizer)

# Data Collation Function
def collate_fn(batch):
    input_ids = torch.stack([item["input_ids"] for item in batch])
    attention_mask = torch.stack([item["attention_mask"] for item in batch])
    labels = torch.tensor([item["label"] for item in batch])
    return {"input_ids": input_ids, "attention_mask": attention_mask, "label": labels}

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=32, collate_fn=collate_fn)

# BioBERT Model with Classification Head
class BioBERTForSpecialistClassification(nn.Module):
    def __init__(self, pretrained_model_name, num_labels, dropout_rate=0.3):
        super(BioBERTForSpecialistClassification, self).__init__()
        self.biobert = AutoModel.from_pretrained(pretrained_model_name)
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(self.biobert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.biobert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # Use the [CLS] token for classification
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

# Initialize the model
model = BioBERTForSpecialistClassification(
    pretrained_model_name=model_name,
    num_labels=len(specialist_mapping),
    dropout_rate=0.3
).to(device)

# Optimizer Setup
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Loss Function
criterion = nn.CrossEntropyLoss()

# Mixed Precision Training
scaler = GradScaler()

# Training Function
def train_model(model, train_loader, optimizer, criterion, device, scaler):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        # Mixed precision training
        with autocast():  # Automatically cast operations to half precision
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = criterion(outputs, labels)

        # Scaler for mixed precision
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        total_loss += loss.item()
    return total_loss / len(train_loader)

# Evaluation Function
def evaluate_model(model, test_loader, device):
    model.eval()
    preds = []
    labels = []
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels.extend(batch["label"].tolist())

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds.extend(torch.argmax(outputs, dim=1).tolist())
    accuracy = accuracy_score(labels, preds)
    return accuracy

# Training Loop
for epoch in range(100):  # Train for 100 epochs
    train_loss = train_model(model, train_loader, optimizer, criterion, device, scaler)
    accuracy = evaluate_model(model, test_loader, device)
    print(f"Epoch {epoch + 1}/50 | Train Loss: {train_loss:.4f} | Test Accuracy: {accuracy:.4f}")

# Save the Fine-Tuned Model
torch.save(model.state_dict(), "./biobert_fine_tuned_model.pth")
tokenizer.save_pretrained("./biobert_fine_tuned_model_tokenizer")

/tmp/ipykernel_171792/1294174196.py:112: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/tmp/ipykernel_171792/1294174196.py:125: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Automatically cast operations to half precision


Epoch 1/5 | Train Loss: 1.3127 | Test Accuracy: 0.6671


/tmp/ipykernel_171792/1294174196.py:125: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Automatically cast operations to half precision


Epoch 2/5 | Train Loss: 1.2185 | Test Accuracy: 0.6184


/tmp/ipykernel_171792/1294174196.py:125: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Automatically cast operations to half precision


...

/tmp/ipykernel_171792/1294174196.py:125: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Automatically cast operations to half precision


Epoch 49/5 | Train Loss: 2.8716 | Test Accuracy: 0.1466


/tmp/ipykernel_171792/1294174196.py:125: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Automatically cast operations to half precision


Epoch 50/5 | Train Loss: 2.8711 | Test Accuracy: 0.1466


('./biobert_fine_tuned_model_tokenizer/tokenizer_config.json',
 './biobert_fine_tuned_model_tokenizer/special_tokens_map.json',
 './biobert_fine_tuned_model_tokenizer/vocab.txt',
 './biobert_fine_tuned_model_tokenizer/added_tokens.json',
 './biobert_fine_tuned_model_tokenizer/tokenizer.json')

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch.cuda.amp import GradScaler, autocast  # For mixed precision training
import pandas as pd

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load CSV
data = pd.read_csv("filtered_specialists.csv")

# Calculate the frequency of each speciality
speciality_counts = data['Specialist'].value_counts()

# Get specialities that occur more than 1000 times
specialities_to_keep = speciality_counts[speciality_counts > 500].index

# Filter the data to keep only rows with specialities that occur more than 1000 times
data = data[data['Specialist'].isin(specialities_to_keep)]

# Map the filtered specialists to numeric labels
specialist_mapping = {spec: idx for idx, spec in enumerate(data["Specialist"].unique())}
data["Specialist_id"] = data["Specialist"].map(specialist_mapping)




In [7]:
data.shape

(87586, 5)

In [8]:
specialities_to_keep

Index(['family medicine', 'obstetrics and gynecology', 'dermatology',
       'psychiatry', 'neurology', 'orthopaedic surgery', 'cardiology',
       'emergency medicine', 'pediatrics', 'urology', 'pulmonology',
       'pathology', 'general surgery', 'sexologist',
       'colon and rectal surgery', 'gastroenterology', 'hospital medicine',
       'vascular surgery', 'oncology', 'nephrology', 'neurological surgery',
       'allergy and immunology', 'endocrinology', 'rheumatology',
       'physical medicine and rehabilitation'],
      dtype='object', name='Specialist')